The notebooks shows how to generate synthetic spike trains.

In [1]:
import numpy as np
import pandas as pd
import pickle
from statsmodels.tsa.stattools import acf
from datetime import datetime

from isttc.spike_utils import simulate_hawkes_thinning, get_trials, bin_trials, bin_spike_train_fixed_len

In [ ]:
log_every = 10

fs = 1000
n_spike_trains = 100
n_trials = 40  
m_iterations = 1 # n samples of trials
bin_size = int(50 * (fs / 1000)) # 50 ms bin
trial_len = int(1000 * (fs / 1000)) # 1000 ms trial

### Generate continuous spike trains

In [ ]:
rate_hz     = 2     # target firing rate (Hz)
tau_ms      = 100.0    # desired autocorr time constant (ms)
alpha       = 0.7     # self‐excitation weight (must be <1)
duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
all_spike_trains = []
for trial in range(n_spike_trains):
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial
    )
    all_spike_trains.append(spikes)


file_name = 'spike_trains.pkl'
data = {'spike_trains': all_spike_trains,
        'alphas': alpha,
        'fr_values': rate_hz,
        'tau_ms': tau_ms,
        'duration_ms': duration_ms}
with open(file_name, 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f'n spike trains {len(all_spike_trains)}, len {duration_ms}')

### Generate trials

In [ ]:
trial_dict = {}
trial_binned_dict = {}

for unit_idx, spikes in enumerate(all_spike_trains):
    if unit_idx % log_every == 0:
        print(f'Processing unit {unit_idx}')

    unit_trial_dict = {}
    unit_trial_binned_dict = {}

    spikes_array = np.asarray([int(spike) for spike in spikes])
    for m in range(m_iterations):
        trials = get_trials(spikes_array, duration_ms, n_trials, trial_len, verbose_=False)
        trials_binned = bin_trials(trials, trial_len, bin_size)

        unit_trial_dict[m] = trials
        unit_trial_binned_dict[m] = trials_binned

    trial_dict[unit_idx] = unit_trial_dict
    trial_binned_dict[unit_idx] = unit_trial_binned_dict


trial_dict_full = {'trial_dict': trial_dict, 
       'n_trials': np.ones(len(trial_dict)).astype(int)*n_trials, 
       'trial_lens': np.ones(len(trial_dict)).astype(int)*trial_len}

trial_binned_dict_full = {'trial_dict': trial_binned_dict, 
                   'n_trials': np.ones(len(trial_binned_dict)).astype(int)*n_trials, 
                   'trial_lens': np.ones(len(trial_binned_dict)).astype(int)*trial_len}

with open('trials.pkl', 'wb') as f:
    pickle.dump(trial_dict_full, f)

with open('trials_binned.pkl', 'wb') as f:
    pickle.dump(trial_binned_dict_full, f)